In [15]:
import multiprocessing
import itertools
import requests
import pandas as pd
from polygon import RESTClient
import pandas as pd
from polygonAPIkey import polygonAPIkey
from pandas_datareader import data as pdr
from tqdm.auto import tqdm
import multiprocessing as mp
import numpy as np
import yfinance as yf
import time
import pickle
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt

yf.pdr_override()

In [16]:
# def tick_cik_table():
#     cik_url = "https://www.sec.gov/files/company_tickers.json"
#     cik_request = requests.get(cik_url, headers={'User-Agent': 'Mozilla/5.0'})
#     companyData = pd.DataFrame.from_dict(cik_request.json(), orient='index')
#     return companyData
   
# cik_df = tick_cik_table()
# cik_df.head()

In [17]:
# def sec_filings(tick, cik_df):
#     cik = cik_df[cik_df['ticker'] == tick]['cik_str'].values[0]
#     #fill leading zeros to 10 digits
#     cik = str(cik)
#     cik = cik.zfill(10)
#     api_url = f"https://data.sec.gov/submissions/CIK{cik}.json"
#     response = requests.get(api_url, headers={'User-Agent': 'Mozilla/5.0'})
#     data = response.json()
#     allForms = pd.DataFrame.from_dict(data['filings']['recent'])
#     allForms = allForms[['reportDate', 'filingDate','form']]
#     # get 10-Q and 10-K
#     allForms = allForms[allForms['form'].isin(['10-Q', '10-K'])]
#     return allForms
    
# sec_filings('TSLA', cik_df)

In [18]:
# #1 sec Q1-4 filings
# #这个api日期还是不准啊 有的差一天 有的差好几天
# def filling_dates_gen(tick, c):
#     dates = []
#     if c == 5:
#         time.sleep(65)
#         c = 0
#     api_url = f"https://api.polygon.io/vX/reference/financials?ticker={tick}&limit=12&sort=filing_date&apiKey=x55vleWy_ZA83DpRkMHG2yfKCvtX2ekp"
#     c +=1 
#     response = requests.get(api_url)
#     print(response.json())
#     # if 'results' not in response.json():
#     #     return
#     for res in response.json()['results']:
#         try:
#             dates.append(pd.to_datetime(res['filing_date'])) if res['timeframe'] == 'quarterly' else ''
#         except:
#             continue
#     dates.sort()
#     return dates, c
 
# filling_dates_gen('TSLA', 0)[0]

In [19]:
earnings = pd.read_csv('earnings.csv')
# unifromize the date format to timestamp
earnings['date'] = pd.to_datetime(earnings['date'])



In [20]:
def df_generator(tick, earning_df):
    
    # 用market date
    # stock rets
    assets_stocks_rets = []

    price = pdr.get_data_yahoo(tick)[['Adj Close', 'Volume']]
    if price.empty:
        return pd.DataFrame()
    mkt_dates = price.index
    price['rets'] = price['Adj Close'].pct_change()

    # get earnings dates
    data = earning_df[earning_df['act_symbol'] == tick]
    today = datetime.today()
    

    # 2 years free api limit
    data = data[(data['date'] <= today) & (data['date'] >= today - timedelta(days=366*2))]

    dates = []
    for i in range(len(data)):
        if data.iloc[i]['when'] == 'After market close':
            after_days = mkt_dates[mkt_dates > data.iloc[i]['date']]
            if len(after_days) == 0:
                continue
            dates.append(after_days[0])
        elif data.iloc[i]['when'] == 'Before market open':
            dates.append(data.iloc[i]['date'])
        else:
            continue
    
    if not dates:
        return pd.DataFrame()
    
    for i in range(len(dates)):
        
        if dates[i] not in mkt_dates:
            
            # company can file reports on holiday, but the date would be display at next business day, so if it's not in mktdates, something is wrong
            return pd.DataFrame()
        
        assets_stocks_rets.append(price.loc[dates[i]].rets)

    assets_stocks_rets = pd.DataFrame(assets_stocks_rets)
    assets_stocks_rets.index = dates
    assets_stocks_rets.columns = ['stocks ret']

    #options rets
    # rest one minute to avoid api limit for every 4 calls, so every 2 loops
    asset_options_rets = pd.DataFrame(index=dates, columns=['ret_c'])


    
    for date in dates[::-1]:

        
        
        # find the closest friday after the date
        wk_day = date.weekday()
        exp_date = date + timedelta(days=4 - wk_day)
     
        # mkt_dates didnt include future dates, fine for now, just backtest after the most recent friday business day. so it can be included
        # alt sol: use futu api, need to build environment, complicated
        while exp_date not in mkt_dates and exp_date <= mkt_dates[-1]:
          
            exp_date += timedelta(days=7)
            
        exp_date = exp_date.strftime('%Y%m%d')
        
        
        prev_day = mkt_dates[mkt_dates < date][-1]
        
        date = date.strftime('%Y-%m-%d')
        prev_day = prev_day.strftime('%Y-%m-%d')
        
        prev_day_price = price.loc[prev_day, 'Adj Close']
        prev_day_vol = price.loc[prev_day, 'Volume']
        
        # check volatility for options
        if prev_day_price < 5 or prev_day_vol < 4000000:
            continue
        
        
        start = prev_day
        end = date
        
        
        
        
        # put option
        # option_type = 'P'
        # strike_p = str(prev_day_price // 5 * 5).split('.')[0]
        # strike_p = str(strike_p).zfill(5)
        
        # contract_name_p = tick + exp_date[2:] + option_type + strike_p + '000'
        # contract_name_p = 'O:' + contract_name_p

        # option_url_p = "https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?adjusted=true&sort=asc&limit=120&apiKey=x55vleWy_ZA83DpRkMHG2yfKCvtX2ekp".format(
        #     contract_name_p, start, end)


        # response_p = requests.get(option_url_p).json()

        
        # # call option
        option_type = 'C'
        strike_c = str((prev_day_price + 5)// 5 * 5).split('.')[0]
        strike_c = str(strike_c).zfill(5)
        
        contract_name_c = tick + exp_date[2:] + option_type + strike_c + '000'
        contract_name_c = 'O:' + contract_name_c

        option_url_c = "https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?adjusted=true&sort=asc&limit=120&apiKey=x55vleWy_ZA83DpRkMHG2yfKCvtX2ekp".format(
            contract_name_c, start, end)
      

        response_c = requests.get(option_url_c).json()

       
            
        try:   
            # ret_p = response_p['results'][1]['o'] / response_p['results'][0]['c'] - 1
            ret_c = response_c['results'][1]['o'] / response_c['results'][0]['c'] - 1
            
            # asset_options_rets.loc[date, 'ret_p'] = ret_p
            asset_options_rets.loc[date, 'ret_c'] = ret_c
        except:
            continue
    
    
    combine_ret = pd.concat([assets_stocks_rets, asset_options_rets], axis=1)
    combine_ret = combine_ret.dropna()
    return combine_ret # columns: stocks ret, ret_p, ret_c




In [21]:
tickers = pd.read_csv('tickers.csv')



dataset = pd.DataFrame(columns=['stocks ret', 'ret_c'])
for tick in tqdm(tickers['ticker']):

    res = df_generator(tick, earnings)
    dataset = dataset.append(res, ignore_index=True)
    
dataset.to_csv('dataset_buy_s_sell_call.csv', index=False)

  0%|          | 0/2014 [00:00<?, ?it/s]

[*********************100%%**********************]  1 of 1 completed

  0%|          | 1/2014 [00:00<21:15,  1.58it/s]


[*********************100%%**********************]  1 of 1 completed

  0%|          | 2/2014 [00:01<16:25,  2.04it/s]

...

100%|█████████▉| 2012/2014 [25:45<00:01,  1.37it/s]

[*********************100%%**********************]  1 of 1 completed

100%|█████████▉| 2013/2014 [25:45<00:00,  1.69it/s]


[*********************100%%**********************]  1 of 1 completed

100%|██████████| 2014/2014 [25:45<00:00,  1.30it/s]